In [ ]:
# %% 
# Imports

import os
import redis
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import subprocess
import time

from brand.timing import timespec_to_timestamp, timeval_to_timestamp

In [ ]:
# %% 
# Start Redis 

SAVE_DIR = '/samba/data/sim/2023-11-17/RawData'
RDB_DIR = os.path.join(SAVE_DIR,'RDB')
RDB_FILENAME = 'sim_231117_012.rdb'
REDIS_IP = '127.0.0.1'
REDIS_PORT = 18000

redis_command = ['/home/snel/Projects/emory-cart/brand/bin/redis-server', '--bind', REDIS_IP, '--port', str(REDIS_PORT)]
redis_command.append('--dbfilename')
redis_command.append(RDB_FILENAME)
redis_command.append('--dir')
redis_command.append(RDB_DIR)

print('Starting redis: ' + ' '.join(redis_command))

proc = subprocess.Popen(redis_command, stdout=subprocess.PIPE)
redis_pid = proc.pid

try:
    out, _ = proc.communicate(timeout=1)
    if out:
        print(out.decode())
    if 'Address already in use' in str(out):
        print("Could not run redis-server (address already in use). Check if a Redis server is already running on that port. Aborting.")
        exit(1)
    else:
        print("Launching redis-server failed for an unknown reason, check supervisor logs. Aborting.")
        exit(1)
except subprocess.TimeoutExpired:  # no error message received
    print('Redis-server is running.')

r = redis.Redis(host=REDIS_IP, port=REDIS_PORT)

busy_loading = True
while busy_loading:
    try:
        print(f"Streams in database: {r.keys('*')}")
        busy_loading = False
    except redis.exceptions.BusyLoadingError:
        print('Redis is busy loading dataset in memory')
        time.sleep(1)

In [ ]:
streams = ['nsp_neural', 'thresh_cross_1', 'sbp_1', 'binned_spikes']
msg = 'Stream Info'
print(msg + '\n' + '-' * len(msg))
for stream in streams:
    n_entries = r.xlen(stream)
    if n_entries > 0:
        entry_dict = r.xrevrange(stream, count=1)[0][1]
        has_sync = True if b'sync' in entry_dict else False
        if has_sync:
            entry_dict
    else:
        has_sync = False

    row = f'{stream :24s}: {n_entries :6d}'
    if has_sync:
        row += f"\tsync={json.loads(entry_dict[b'sync'])}"
    else:
        row += '\tsync=None'
    print(row)

In [ ]:
# utility function

def scalarfrombuffer(*args, **kwargs):
    return np.frombuffer(*args, **kwargs)[0]

samp_per_packet = 30

# build dataframe with neural data

stream = 'nsp_neural'
entries = r.xrange(stream)
data = [None] * len(entries)
for i, (_, entry_data) in enumerate(entries):
    data[i] = {
        f: entry_data[f.encode()]
        for f in ['timestamps', 'BRANDS_time']
    }
    brands_time = (
        np.array(timespec_to_timestamp(entry_data[b'BRANDS_time'])) *
        1e9).astype(np.uint64)
    udp_time = (np.array(timeval_to_timestamp(entry_data[b'udp_recv_time'])) *
                1e9).astype(np.uint64)
    nsp_idx = np.frombuffer(entry_data[b'timestamps'], dtype=np.uint64)
    # samples = np.frombuffer(entry_data[b'samples'], dtype=np.int16).reshape((-1,samp_per_packet)).T
    samples = np.reshape(np.frombuffer(entry_data[b'samples'], dtype=np.int16),(samp_per_packet, -1))
    data[i] = {
        'ts': brands_time,
        'sync': nsp_idx[0],
        'sync_all': nsp_idx,
        'udp_in': udp_time,
        'samples': samples,
        'timestamps': nsp_idx,
    }
df_ca = pd.DataFrame(data)
df_ca.set_index('sync',inplace=True,drop=True)
print(df_ca)



In [ ]:
# print ISI for cerebusAdapter data

isi = np.diff(df_ca.index.values) / 1e6
print(f'cerebusAdapter (sync)   ISI: {isi.mean():2.4f} +- {isi.std():2.4f}'
      f' ({isi.min():2.4f} - {isi.max():2.4f}) ms')

isi = df_ca['ts'].diff()[1:].values / 1e6
print(f'cerebusAdapter (ts)     ISI: {isi.mean():2.4f} +- {isi.std():2.4f}'
      f' ({isi.min():2.4f} - {isi.max():2.4f}) ms')

isi = df_ca['udp_in'].diff()[1:].values / 1e6
print(f'cerebusAdapter (udp ts) ISI: {isi.mean():2.4f} +- {isi.std():2.4f}'
      f' ({isi.min():2.4f} - {isi.max():2.4f}) ms')

In [ ]:
# stack stream entries to build array with all continuous data

cont_data = np.vstack(df_ca.samples.values)
print(cont_data.shape)

# plot continuous data

crange = np.arange(90,100,1)
n_channels = crange.shape[0]
trange = slice(0, 30 * 1000)

fig, axes = plt.subplots(ncols=1,
                         nrows=n_channels,
                         figsize=(10, n_channels * 2),
                         sharey=False,
                         sharex=True,
                         facecolor='w')
for ich, ch in enumerate(crange):
    ax = axes.flat[ich]
    ax.plot(cont_data[trange, ch])
    ax.set_title(f'Ch {ch}')

In [ ]:
stream = 'thresh_cross_1'
entries = r.xrange(stream)
data = [None] * len(entries)
for i, (_, entry_data) in enumerate(entries):
    data[i] = {
        f: entry_data[f.encode()]
        for f in ['timestamps']
    }
    ts = np.frombuffer(entry_data[b'ts'], dtype=np.uint64)[0]
    nsp_idx = np.frombuffer(entry_data[b'timestamps'], dtype=np.uint64)[0]
    # samples = np.frombuffer(entry_data[b'samples'], dtype=np.int16).reshape((-1,samp_per_packet)).T
    samples = np.frombuffer(entry_data[b'crossings'], dtype=np.int16),(-1, samp_per_packet)
    data[i] = {
        'ts': ts,
        'sync': nsp_idx,
        'samples': samples,
    }
df_tc = pd.DataFrame(data)
df_tc.set_index('sync',inplace=True,drop=True)
print(df_tc)

In [ ]:
# print ISI for threshold_crossings data

isi = np.diff(df_tc.index.values) / 1e6
print(f'thresh_cross (sync)   ISI: {isi.mean()} +- {isi.std()}'
      f' ({isi.min()} - {isi.max()}) ms')

isi = df_tc['ts'].diff()[1:].values / 1e6
print(f'thresh_cross (ts)     ISI: {isi.mean()} +- {isi.std()}'
      f' ({isi.min()} - {isi.max()}) ms')

In [ ]:
df = df_ca.join(df_tc, lsuffix='_ca', rsuffix='_tc')
df.dropna(inplace=True)
print(df.columns)
print(df[['ts_tc', 'ts_ca']])

In [ ]:
latency_tc = df['ts_tc'] - df['ts_ca']

plt.plot(latency_tc / 1e6)
plt.show()

In [ ]:
plt.plot(df['ts_ca'].diff()[1:].values / 1e6)
plt.show()

In [ ]:
r.shutdown(nosave=True)